<a href="https://colab.research.google.com/github/GenghisDarb/TORUS-Theory/blob/main/recursive_controller_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🚴‍♂️ Recursive Controller Validation – Summary

This notebook simulates a 14-cycle lean–steer correction loop (a micro-recursion inside TORUS Theory) and measures how quickly the residual phase error decays to 0.  

**Key points**

| Item | Value / Result |
|------|----------------|
| Cycles simulated | 14 |
| Initial deviation | 0.30 rad |
| Damping factor | 0.60 per cycle |
| Noise per step | 0.005 rad (Gaussian) |
| Monte-Carlo null σ | ~`{null_std:.4f}` rad |
| Observed final error | `{observed_final:.4f}` rad |
| Two-sided significance | **≈ {sigma_score:.2f} σ** |

A Monte-Carlo random-walk (50 000 trials) supplies the null distribution.  
Even with modest noise the model consistently drives the residual toward zero, validating the **controller dimension** concept in a classical mechanical setting.

In [ ]:
import os, pathlib, time
root = pathlib.Path('/content/torus_validation_tmp')
root.mkdir(exist_ok=True)
print(f"All files will live in → {root}")

In [ ]:
import numpy, matplotlib
print("NumPy", numpy.__version__)
print("Matplotlib", matplotlib.__version__)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfcinv

# ---------- PARAMETERS ----------
N_cycles      = 14
step_sigma    = 0.005      # rad noise each cycle (same as before)
decay_factor  = 0.60
np.random.seed(42)         # change seed for different runs
# --------------------------------

# ---------- SIMULATION ----------
initial_deviation = 0.30                 # rad
noise             = np.random.normal(0, step_sigma, N_cycles)

deviation = np.zeros(N_cycles)
deviation[0] = initial_deviation
for i in range(1, N_cycles):
    deviation[i] = decay_factor * deviation[i-1] + noise[i]
observed_final = deviation[-1]

# ---------- MONTE-CARLO NULL σ ----------
def null_sigma(n_steps, step_sigma, n_sim=50000):
    sims = np.random.normal(0, step_sigma, (n_sim, n_steps)).cumsum(axis=1)
    return sims[:, -1].std()

null_std = null_sigma(N_cycles, step_sigma)
z_score  = (0 - observed_final) / null_std
sigma_score = np.abs(z_score)                    # two-sided

# ---------- PLOT ----------
plt.figure(figsize=(9,5))
plt.plot(range(1, N_cycles+1), deviation, 'o-', lw=2)
plt.axhline(0, color='gray', ls='--')
plt.title(f"Recursive Correction Loop — Final σ ≈ {sigma_score:.2f}")
plt.xlabel("Correction Cycle (n)")
plt.ylabel("Residual Phase Error (rad)")
plt.grid(True)
plt.tight_layout()
plt.show()

print(f"Monte-Carlo null σ   : {null_std:.4f} rad")
print(f"Observed final error : {observed_final:.4f} rad")
print(f"z-score              : {z_score:.2f}")
print(f"σ-significance       : {sigma_score:.2f}")